In [1]:
import pandas as pd
import random
import logging
import openai
from bertopic import BERTopic
from bertopic.representation import OpenAI
from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.decomposition import PCA
from bertopic.dimensionality import BaseDimensionalityReduction
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

/home/jovyan/virtualENV/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-28 15:43:13.605761: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-28 15:43:13.648091: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-28 15:43:13.648126: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-28 15:43:13.649180: 

In [2]:
df = pd.read_csv("training-data.csv")
docs = list(df["Text"])
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
summarization_prompt = """
I have a topic that is described by the following keywords: [KEYWORDS]
In this topic, the following documents are a small but representative subset of all documents in the topic:
[DOCUMENTS]

Based on the information above, please give a sentence description of this topic in the following format:
<description>
"""

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

# Train BERTopic with a custom CountVectorizer
vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words="english")
ctfidf_model = ClassTfidfTransformer(bm25_weighting=True)
# Create your representation model
client = openai.OpenAI(api_key="API_KEY")
representation_model = OpenAI(client, model="gpt-4-0125-preview", delay_in_seconds=2, chat=True, nr_docs=5, prompt=summarization_prompt)
topic_model = BERTopic(verbose=True, ctfidf_model=ctfidf_model, embedding_model="all-mpnet-base-v2", vectorizer_model=vectorizer_model)
topics, probs = topic_model.fit_transform(docs)

2024-03-28 15:45:27,957 - BERTopic - Embedding - Transforming documents to embeddings.
2024-03-28 15:45:27,958 - INFO - Load pretrained SentenceTransformer: all-mpnet-base-v2
2024-03-28 15:45:29,835 - INFO - Use pytorch device_name: cuda
Batches: 100%|██████████| 122/122 [00:02<00:00, 46.68it/s]
2024-03-28 15:45:32,929 - BERTopic - Embedding - Completed ✓
2024-03-28 15:45:32,930 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-28 15:46:04,337 - BERTopic - Dimensionality - Completed ✓
2024-03-28 15:46:04,339 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-03-28 15:46:04,484 - BERTopic - Cluster - Completed ✓
2024-03-28 15:46:04,495 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-03-28 15:46:04,945 - BERTopic - Representation - Completed ✓


In [7]:
new_topics = topic_model.reduce_outliers(docs, topics, strategy="embeddings")

In [10]:
topic_model.update_topics(docs, topics=new_topics, representation_model=representation_model)

2024-03-28 15:48:15,656 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings will also be created through weightedc-TF-IDF embeddings instead of centroid embeddings.
100%|██████████| 86/86 [07:40<00:00,  5.35s/it]


In [13]:
df_topics = topic_model.get_document_info(docs)

In [18]:
df_topics["Sentiment"] = df["Sentiment"]

In [21]:
df_topics.to_csv("topic-data.csv", index=False)